In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Train and Test sets

## Training set (imputing and normalizing)

In [2]:
df = pd.read_excel("3.xlsx")

In [3]:
df.head()

,Patient_Hospital,Outcome_InhospitalMortality,Outcome_ICUadmission,TM_S_Intubation,TM_S_Dialysis,L1_BloodGroup_First,Demographic_Gender,Symptom_Caugh,Symptom_Dyspnea,Symptom_Fever,...,LAB_CR_1,LAB_NA_First,LAB_K_First,LAB_ALKP_First,LAB_ESR_First,LAB_CPK_First,LAB_PTT_First,LAB_PT_First,LAB_INR_First,Demographic_Age
0,Hospital1,0.0,1,1,1,0.0,1.0,1,1,0,...,NaN,140.0,4.0,181.0,62.0,661.0,32.6,10.9,1.00,33.0
1,Hospital1,0.0,1,1,0,NaN,1.0,1,1,0,...,1.1,139.0,3.5,98.0,30.0,179.0,37.6,10.8,0.99,44.0
2,Hospital1,0.0,1,0,1,NaN,1.0,0,1,0,...,0.9,138.0,4.2,99.0,29.0,1956.0,27.4,12.1,1.11,50.0
3,Hospital1,1.0,1,0,0,NaN,1.0,0,1,1,...,1.2,137.0,4.7,139.0,20.0,30.0,29.1,14.2,1.31,71.0
4,Hospital1,1.0,1,0,0,NaN,1.0,0,1,1,...,1.2,137.0,4.7,139.0,20.0,30.0,29.1,14.2,1.31,71.0


In [4]:
# ooutcome columns: 1.are Outcome_InhospitalMortality 
df = df.dropna(subset=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])

In [5]:
#inja oomadim va external validation ro freeze kardim
df_internal_validation = df[df['Patient_Hospital']!="Hospital2"]
df_external_validation = df[df['Patient_Hospital']=="Hospital2"]
df_internal_validation = df_internal_validation.drop(columns=['Patient_Hospital'])
df_external_validation = df_external_validation.drop(columns=['Patient_Hospital'])

In [6]:
#inja train test split kardim va target variable ro jodakardim
# Define features (X) and target variable (y)
X = df_internal_validation.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])
y  = df_internal_validation[['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis']]
# Perform the train-test split (adjust the test_size as needed)
X_train , X_test , y_train , y_test = train_test_split(X,y , test_size = 0.2 , random_state = 42)

In [7]:
#inja miaim va data haye categorical va numerical ro moshakhas mikonim ke bade knn categorical ha ro round konim
cat_col=[]
num_col=[]
for i in range(len(X_train.columns)):
    if i <=52:
        cat_col.append(X_train.columns[i])
    else:
        num_col.append(X_train.columns[i])


In [8]:
from sklearn.impute import KNNImputer
X_train_array  =X_train.to_numpy()
imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputed = imputer.fit_transform(X_train_array)

In [9]:
df_imputed = pd.DataFrame(imputed)
df_imputed.columns=pd.DataFrame(X_train).columns.tolist()


In [10]:
for i in cat_col:
    df_imputed[[i]] = df_imputed[[i]].round()

In [11]:
df_imputed.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

In [12]:
#miaim x , y ro be ham michaboonim ta betonim normilize konim
df_before_normalize = pd.concat([df_imputed , y_train], axis=1)

In [13]:
arr_tain =df_imputed.to_numpy()

In [14]:
from sklearn import preprocessing
transform = preprocessing.StandardScaler()
df_normalize = transform.fit_transform(arr_tain)
df_normalize =pd.DataFrame(df_normalize)
df_normalize.columns = df_imputed.columns.tolist()

In [15]:
X_train = df_normalize
y_train = df_before_normalize[['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis']]

In [16]:
y.value_counts("Outcome_InhospitalMortality")

Outcome_InhospitalMortality
1.0    6115
0.0    1533
Name: count, dtype: int64

In [17]:
y.value_counts("Outcome_ICUadmission")

Outcome_ICUadmission
0    6387
1    1261
Name: count, dtype: int64

## Test set (imputing and normalizing)

In [18]:
#inja miaim va data haye categorical va numerical ro moshakhas mikonim ke bade knn categorical ha ro round konim
cat_col=[]
num_col=[]
for i in range(len(X_test.columns)):
    if i <=52:
        cat_col.append(X_test.columns[i])
    else:
        num_col.append(X_test.columns[i])


In [19]:
from sklearn.impute import KNNImputer
X_test_array  =X_test.to_numpy()
imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputed_test = imputer.fit_transform(X_test_array)

In [20]:
df_imputed_test = pd.DataFrame(imputed_test)
df_imputed_test.columns=pd.DataFrame(X_test).columns.tolist()


In [21]:
for i in cat_col:
    df_imputed_test[[i]] = df_imputed_test[[i]].round()

In [22]:
df_imputed_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [23]:
#miaim x , y ro be ham michaboonim ta betonim normilize konim
df_test_before_normalize = pd.concat([df_imputed_test , y_test], axis=1)

In [24]:
arr =df_test_before_normalize.to_numpy()

In [25]:
from sklearn import preprocessing
transform = preprocessing.StandardScaler()
df_test_normalize = transform.fit_transform(arr)
df_test_normalize =pd.DataFrame(df_test_normalize)
df_test_normalize.columns = df_internal_validation.columns.tolist()
df_test_normalize

,Outcome_InhospitalMortality,Outcome_ICUadmission,TM_S_Intubation,TM_S_Dialysis,L1_BloodGroup_First,Demographic_Gender,Symptom_Caugh,Symptom_Dyspnea,Symptom_Fever,Symptom_Chiver,...,LAB_CR_1,LAB_NA_First,LAB_K_First,LAB_ALKP_First,LAB_ESR_First,LAB_CPK_First,LAB_PTT_First,LAB_PT_First,LAB_INR_First,Demographic_Age
0,0.273437,0.923201,1.015811,0.858291,1.149652,-0.622195,1.628358,-0.760886,-0.303442,-0.128885,...,0.197369,-0.372103,-0.134941,-0.054545,-0.167263,-0.319589,0.488740,-0.29173,-0.439827,-0.172075
1,-0.826059,-1.083188,-0.984435,-1.165106,-0.869828,-0.622195,-0.614115,-0.760886,3.295522,-0.128885,...,-0.098694,-0.247141,-0.267685,-0.231803,-0.178906,0.722315,0.488740,-0.29173,-0.439827,-0.172075
2,-0.276311,-1.083188,1.015811,0.858291,1.149652,1.607214,-0.614115,-0.760886,-0.303442,-0.128885,...,0.157894,-0.291846,-0.267685,-0.268380,-0.187988,0.886826,0.488740,-0.29173,-0.439827,-0.172075
3,1.372933,0.923201,-0.984435,-1.165106,1.149652,1.607214,-0.614115,-0.760886,-0.303442,-0.128885,...,-0.197382,-0.113379,-0.008690,-0.199915,-0.190549,-0.264752,0.488740,-0.29173,-0.439827,-0.172075
4,0.273437,0.923201,-0.984435,0.858291,-0.869828,-0.622195,1.628358,-0.760886,-0.303442,-0.128885,...,-1.036230,-0.088739,-0.088048,-0.153022,-0.174249,-0.484100,0.488740,-0.29173,-0.439827,-0.172075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,-1.375807,0.923201,1.015811,0.858291,1.149652,1.607214,1.628358,-0.760886,-0.303442,-0.128885,...,0.325664,-0.162660,-0.199870,-0.204605,-0.179605,0.777152,0.488740,-0.29173,-0.439827,-0.172075
1526,-0.826059,0.923201,-0.984435,0.858291,-0.869828,-0.622195,-0.614115,-0.760886,-0.303442,-0.128885,...,-0.453971,-0.122883,-0.299789,-0.247747,-0.184727,1.325522,-2.046079,-0.29173,-0.439827,-0.172075
1527,0.823185,-1.083188,1.015811,0.858291,1.149652,-0.622195,-0.614115,1.314257,-0.303442,-0.128885,...,-1.184262,-0.041218,-0.264799,-0.312460,-0.192295,0.886826,0.488740,-0.29173,-0.439827,-0.172075
1528,2.472429,0.923201,-0.984435,0.858291,1.149652,1.607214,1.628358,-0.760886,-0.303442,-0.128885,...,-0.532921,-0.278821,-0.044762,-0.199915,-0.190549,-0.484100,0.488740,-0.29173,2.273622,-0.172075


In [26]:
X_test = df_test_normalize.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])
y_test = df_test_before_normalize[['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis']]

>>>Now dataset is cleaned and normalized 

In [27]:
y_train_Mortality =pd.DataFrame(y_train.loc[ : ,'Outcome_InhospitalMortality'])

In [28]:
y_test_Mortality =pd.DataFrame(y_train.loc[ : ,'Outcome_InhospitalMortality'])

## data selection Mortality and ICU

In [29]:
from sklearn.linear_model import Lasso

def lasso_MO_ICU(X_train, y_train, X_test):
    # Perform feature selection using LASSO on the training set
    lasso = Lasso(alpha=0.1)
    lasso.fit(X_train, y_train)

    # Get the absolute coefficients and sort them
    absolute_coeffs = np.abs(lasso.coef_)
    sorted_indices = np.argsort(absolute_coeffs)[::-1]

    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)

    # Select the top 2 features with the highest coefficients
    selected_feature_indices = sorted_indices[:40]

    # Apply feature selection to both the training and test sets
    X_train_selected_Mortality_ICU = X_train.iloc[:, selected_feature_indices]
    X_test_selected_Mortality_ICU = X_test.iloc[:, selected_feature_indices]

    return X_train_selected_Mortality_ICU, X_test_selected_Mortality_ICU


# Undersampling and and Oversampling


In [30]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTEENN
from imblearn.ensemble import RUSBoostClassifier
def balancing_dataset(X_train,y_train , X_test , y_test):
    
    tl = TomekLinks()
    X_resampled_train , y_resampled_train = tl.fit_resample(X_train,y_train)
    X_resampled_test , y_resampled_test = tl.fit_resample(X_test,y_test)
    
    return (X_resampled_train, y_resampled_train,X_resampled_test , y_resampled_test )

# experimenting different models on dataset

In [31]:

import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [32]:
X_train = np.array(X_train)
X_test = np.array(X_test)

## Logistic Regression

In [33]:

#remember to consider x_test and y_test for each model

In [34]:
y_train_Outcome_InhospitalMortality = np.array(y_train['Outcome_InhospitalMortality'].tolist())
y_test_Outcome_InhospitalMortality = np.array(y_test['Outcome_InhospitalMortality'].tolist())

In [35]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)
def logistic_regression_classifier(X_train, y_train, X_test, y_test):

    parameters_lr = {}


    lr = LogisticRegression(random_state=12345)


    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_predicted_Mortality_lr = best_classifier_lr.predict(X_test)

 
    accuracy = accuracy_score(y_test, y_predicted_Mortality_lr)
    cm = confusion_matrix(y_test, y_predicted_Mortality_lr)
    precision = precision_score(y_test, y_predicted_Mortality_lr)
    recall = recall_score(y_test, y_predicted_Mortality_lr)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_lr)


    Logistic_Regression = {}
    Logistic_Regression["accuracy"] = accuracy
    Logistic_Regression["precision"] = precision
    Logistic_Regression["recall"] = recall
    Logistic_Regression["specificity"] = specificity
    Logistic_Regression["f1"] = f1

    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_predicted_Mortality_lr
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


## SVM

In [36]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_svm(X_train, y_train, X_test, y_test):

    svm_classifier = svm.SVC(random_state=1234)


    parameters_svm = {}


    grid_search_svm = GridSearchCV(
        estimator=svm_classifier,
        param_grid=parameters_svm,
        cv=5
    )
    
   
    svm_cv = grid_search_svm.fit(X_train, y_train)

 
    best_classifier_svm = svm_cv.best_estimator_


    y_predicted_Mortality_svm = best_classifier_svm.predict(X_test)

   
    accuracy = accuracy_score(y_test, y_predicted_Mortality_svm)
    cm = confusion_matrix(y_test, y_predicted_Mortality_svm)
    precision = precision_score(y_test, y_predicted_Mortality_svm)
    recall = recall_score(y_test, y_predicted_Mortality_svm)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_svm)

    svm_results = {}
    svm_results["accuracy"] = accuracy
    svm_results["precision"] = precision
    svm_results["recall"] = recall
    svm_results["specificity"] = specificity
    svm_results["f1"] = f1

  
    y_plot = {}
    y_plot["y_true"] = y_test  
    y_plot["y_predicted"] = y_predicted_Mortality_svm

    svm_results = (svm_results, y_plot)
    
    return svm_results



## Decision tree

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_tree(X_train, y_train, X_test, y_test):

    parameters_tree = {}

 
    tree = DecisionTreeClassifier(random_state=12345)

 
    grid_search_tree = GridSearchCV(
        estimator=tree,
        param_grid=parameters_tree,
        cv=5
    )
    

    tree_cv = grid_search_tree.fit(X_train, y_train)

  
    best_classifier_tree = tree_cv.best_estimator_

 
    y_predicted_Mortality_tree = best_classifier_tree.predict(X_test)

 
    accuracy = accuracy_score(y_test, y_predicted_Mortality_tree)
    cm = confusion_matrix(y_test, y_predicted_Mortality_tree)
    precision = precision_score(y_test, y_predicted_Mortality_tree)
    recall = recall_score(y_test, y_predicted_Mortality_tree)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_tree)

  
    tree_results = {}
    tree_results["accuracy"] = accuracy
    tree_results["precision"] = precision
    tree_results["recall"] = recall
    tree_results["specificity"] = specificity
    tree_results["f1"] = f1


    y_plot = {}
    y_plot["y_true"] = y_test 
    y_plot["y_predicted"] = y_predicted_Mortality_tree

    tree_results = (tree_results, y_plot)
    
    return tree_results



## KNN

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_knn(X_train, y_train, X_test, y_test):

    parameters_knn = {}

 
    knn = KNeighborsClassifier()


    grid_search_knn = GridSearchCV(
        estimator=knn,
        param_grid=parameters_knn,
        cv=5
    )
    

    knn_cv = grid_search_knn.fit(X_train, y_train)

    best_classifier_knn = knn_cv.best_estimator_

    y_predicted_Mortality_knn = best_classifier_knn.predict(X_test)

    accuracy = accuracy_score(y_test, y_predicted_Mortality_knn)
    cm = confusion_matrix(y_test, y_predicted_Mortality_knn)
    precision = precision_score(y_test, y_predicted_Mortality_knn)
    recall = recall_score(y_test, y_predicted_Mortality_knn)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_knn)

    knn_results = {}
    knn_results["accuracy"] = accuracy
    knn_results["precision"] = precision
    knn_results["recall"] = recall
    knn_results["specificity"] = specificity
    knn_results["f1"] = f1


    y_plot = {}
    y_plot["y_true"] = y_test  
    y_plot["y_predicted"] = y_predicted_Mortality_knn

    knn_results = (knn_results, y_plot)
    
    return knn_results



## Random forest

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_forest(X_train, y_train, X_test, y_test):
    parameters_forest = {}


    forest = RandomForestClassifier()


    grid_search_forest = GridSearchCV(
        estimator=forest,
        param_grid=parameters_forest,
        cv=5
    )
    

    forest_cv = grid_search_forest.fit(X_train, y_train)

    best_classifier_forest = forest_cv.best_estimator_

    y_predicted_Mortality_forest = best_classifier_forest.predict(X_test)


    accuracy = accuracy_score(y_test, y_predicted_Mortality_forest)
    cm = confusion_matrix(y_test, y_predicted_Mortality_forest)
    precision = precision_score(y_test, y_predicted_Mortality_forest)
    recall = recall_score(y_test, y_predicted_Mortality_forest)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_forest)


    forest_results = {}
    forest_results["accuracy"] = accuracy
    forest_results["precision"] = precision
    forest_results["recall"] = recall
    forest_results["specificity"] = specificity
    forest_results["f1"] = f1

    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_predicted_Mortality_forest

    return forest_results, y_plot


## Neural Network

In [40]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_neural(X_train, y_train, X_test, y_test):

    parameters_neural = {}


    neural = MLPClassifier()

    grid_search_neural = GridSearchCV(
        estimator=neural,
        param_grid=parameters_neural,
        cv=5
    )
    

    neural_cv = grid_search_neural.fit(X_train, y_train)

   
    best_classifier_neural = neural_cv.best_estimator_

    
    y_predicted_Mortality_neural = best_classifier_neural.predict(X_test)

    accuracy = accuracy_score(y_test, y_predicted_Mortality_neural)
    cm = confusion_matrix(y_test, y_predicted_Mortality_neural)
    precision = precision_score(y_test, y_predicted_Mortality_neural)
    recall = recall_score(y_test, y_predicted_Mortality_neural)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_neural)

 
    neural_results = {}
    neural_results["accuracy"] = accuracy
    neural_results["precision"] = precision
    neural_results["recall"] = recall
    neural_results["specificity"] = specificity
    neural_results["f1"] = f1

    
    y_plot = {}
    y_plot["y_true"] = y_test  
    y_plot["y_predicted"] = y_predicted_Mortality_neural

    return neural_results, y_plot



## XGboost

In [41]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_boost(X_train, y_train, X_test, y_test):
  
    parameters_boost = {}

 
    boost = XGBClassifier()


    grid_search_boost = GridSearchCV(
        estimator=boost,
        param_grid=parameters_boost,
        cv=5
    )
    

    boost_cv = grid_search_boost.fit(X_train, y_train)


    best_classifier_boost = boost_cv.best_estimator_


    y_predicted_Mortality_boost = best_classifier_boost.predict(X_test)


    accuracy = accuracy_score(y_test, y_predicted_Mortality_boost)
    cm = confusion_matrix(y_test, y_predicted_Mortality_boost)
    precision = precision_score(y_test, y_predicted_Mortality_boost)
    recall = recall_score(y_test, y_predicted_Mortality_boost)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_boost)

  
    boost_results = {}
    boost_results["accuracy"] = accuracy
    boost_results["precision"] = precision
    boost_results["recall"] = recall
    boost_results["specificity"] = specificity
    boost_results["f1"] = f1


    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_predicted_Mortality_boost

    return boost_results, y_plot



# Final results

In [42]:
list_Outcome_InhospitalMortality=[]


X_train = df_normalize
y_train = df_before_normalize.loc[:,'Outcome_InhospitalMortality']
X_test = df_test_normalize.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])
y_test = df_test_before_normalize.loc[:,'Outcome_InhospitalMortality']
y_train_ = pd.DataFrame(y_train)
y_test_ = pd.DataFrame(y_test)
X_train, X_test = lasso_MO_ICU(pd.DataFrame(X_train), y_train_ ,pd.DataFrame(X_test))


In [43]:
X_train.to_excel("features after selection for LLM model.xlsx", index=False)

In [44]:

X_train=np.array(X_train)
X_test=np.array(X_test)

y_train_ = np.array(y_train)
y_test_ = np.array(y_test)

X_train,y_train_ , X_test , y_test_ = balancing_dataset(X_train,y_train_ , X_test , y_test_)

list_Outcome_InhospitalMortality.extend([logistic_regression_classifier(X_train, y_train_, X_test, y_test_),
                                            train_and_evaluate_svm(X_train, y_train_, X_test, y_test_),
                                        train_and_evaluate_tree(X_train, y_train_, X_test, y_test_),
                                        train_and_evaluate_knn(X_train, y_train_, X_test, y_test_),
                                        train_and_evaluate_forest(X_train, y_train_, X_test, y_test_),
                                        train_and_evaluate_neural(X_train, y_train_, X_test, y_test_),
                                        train_and_evaluate_boost(X_train, y_train_, X_test, y_test_)])


result_dic_list_Outcome_InhospitalMortality = dict(zip(['logistic_regression', 'svm', 'tree', 'knn', 'forest', 'neural', 'boost'], list_Outcome_InhospitalMortality))



merged_dict = {}

# List of dictionary names and their corresponding dictionaries
dict_list = [('Outcome_InhospitalMortality', result_dic_list_Outcome_InhospitalMortality)]

# Merge the dictionaries
for name, result_dict in dict_list:
    merged_dict[name] = result_dict

# The merged_dict now contains all the dictionaries merged together


df = pd.DataFrame(merged_dict)


df = df.transpose()


df.reset_index(inplace=True)
df.rename(columns={'index': 'Method'}, inplace=True)


data = []

for outcome, models in merged_dict.items():
    for model, metrics in models.items():
        accuracy, precision, recall, specificity, f1 = metrics[0]['accuracy'], metrics[0]['precision'], metrics[0]['recall'], metrics[0]['specificity'], metrics[0]['f1']
        y_true, y_predicted = metrics[1]['y_true'], metrics[1]['y_predicted']
        data.append([outcome, model, accuracy, precision, recall, specificity, f1, y_true.tolist(), y_predicted.tolist()])

columns = ['Outcome', 'Model', 'Accuracy', 'Precision', 'Recall', 'Specificity', 'F1', 'y_true', 'y_predicted']

df = pd.DataFrame(data, columns=columns)


d:\python\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\python\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum 

In [45]:
df

,Outcome,Model,Accuracy,Precision,Recall,Specificity,F1,y_true,y_predicted
0,Outcome_InhospitalMortality,logistic_regression,0.800402,0.817797,0.966611,0.966611,0.885998,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1,Outcome_InhospitalMortality,svm,0.753516,0.803807,0.916528,0.916528,0.856474,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, ..."
2,Outcome_InhospitalMortality,tree,0.662425,0.828598,0.730384,0.730384,0.776398,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, ..."
3,Outcome_InhospitalMortality,knn,0.748158,0.805804,0.904007,0.904007,0.852085,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, ..."
4,Outcome_InhospitalMortality,forest,0.698593,0.816949,0.804674,0.804674,0.810765,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, ..."
5,Outcome_InhospitalMortality,neural,0.762224,0.829040,0.886477,0.886477,0.856797,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, ..."
6,Outcome_InhospitalMortality,boost,0.800402,0.808219,0.984975,0.984975,0.887886,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [46]:
df.to_excel('Experiment_1_ML_results.xlsx',index=False)